In [2]:
import requests
import json

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import openweathermapy.core as owm

from config import api_key
from config import gkey

#import time
#from datetime import timedelta
#from datetime import datetime
#from config import gkey

In [3]:
#Input the target city and state
target_city = input("Enter city & state in the format 'city,state: ")

Enter city & state in the format 'city,state: Matawan, NJ


In [4]:
City = target_city.split(",")[0]

In [5]:
#Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

In [6]:
#Creating the params dict with address and key
params = {"address": target_city, "key": gkey}

In [7]:
# Run request (Build URL using the base_url & params)
response = requests.get(base_url, params=params)
#print(response.url)

# Convert the response to JSON
city_geo = response.json()

#print(city_geo)

# Extract Lat/Lng
lat = city_geo["results"][0]["geometry"]["location"]["lat"]
lng = city_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print("%s: %s, %s" % (target_city, lat, lng))

#Converting the lat & lng to string
lat = str(lat)
lng = str(lng)

Matawan, NJ: 40.41482999999999, -74.2295891


In [8]:
#https://developers.google.com/places/web-service/supported_types

#Segregated the places types into indoor and outdoor activities 
Acty_In = pd.read_excel("Resources/Indoor.xlsx")
Acty_Out = pd.read_excel("Resources/Outdoor.xlsx")

#Converting the dataframe into list
Acty_Outlist = Acty_Out["Outdoor"].tolist()
Acty_Inlist = Acty_In["Indoor"].tolist()

In [9]:
#Read activities CSV
Indoor_Outdoor_Weather = "Resources/Indoor_Outdoor_Weather - API Target Names.csv"
activities_df = pd.read_csv(Indoor_Outdoor_Weather, encoding="ISO-8859-1")
activities_df.head()

#Read all Main weather csv files 
main = "Resources/Indoor_Outdoor_Weather - main.csv"
main_df = pd.read_csv(main, encoding="ISO-8859-1")
Atmosphere = "Resources/Indoor_Outdoor_Weather - Atmosphere.csv"
atmosphere_df = pd.read_csv(Atmosphere, encoding="ISO-8859-1")
Drizzle= "Resources/Indoor_Outdoor_Weather - Drizzle.csv"
drizzle_df = pd.read_csv(Drizzle, encoding="ISO-8859-1")
Rain = "Resources/Indoor_Outdoor_Weather - Rain.csv"
rain_df = pd.read_csv(Rain, encoding="ISO-8859-1")
Snow ="Resources/Indoor_Outdoor_Weather - Snow.csv"
snow_df = pd.read_csv(Snow, encoding="ISO-8859-1")


settings = {"units": "metric", "appid": api_key}

#Get current weather
current_weather_newark = owm.get_current(City, **settings)
#print(f"Current weather object for Newark: {current_weather_newark}.")

summary = ["name", 'weather', "main",'clouds']
data = current_weather_newark(*summary)
# print(f"The current weather summary for Newark is: {data}.")
currentmain = data[1][0]["main"]
#print (currentmain)

temp = data[2]['temp']
#print (temp)

humidity = data[2]['humidity']
#print (humidity)

currentdesc = data[1][0]['description']
#print (currentdesc)

# set up lists to hold main description of weather, outdoor weather, indoor weather
# main_description = []
# outdoor = []
# indoor = []

#TEST CASES
# currentmain ="Rain"
# currentdesc = "light rain"
# temp = 35
# humidity =61

finalresult=""
 
# loop through main weather types to determine if outdoor or indoor 
if (currentmain=="Clouds") or (currentmain=="Clear") or (currentmain=="Thunderstorms") or (currentmain=="Mist"):
    if(currentmain=="Thunderstorms"):
        print(f'Staysafe during {currentmain}')
        finalresult="indoor"
    else:        
        print(f'Enjoy during {currentmain}')
        finalresult="outdoor"
else:
    if currentmain=="Rain":
        #read rain.df to find which
        for index, desctype in rain_df.iterrows():
            if currentdesc == desctype["Description"]:
                finalresult=rain_df["Rating"][index]
    elif currentmain=="Snow" :
        #read snow.df to find which
        for index, desctype in snow_df.iterrows():
            if currentdesc == desctype["Description"]:
                finalresult=desctype["Rating"][index]
                
    elif currentmain=="Drizzle" :
        #read drizzledf to find which
        for index, desctype in drizzle_df.iterrrows():
            if currentdesc == desctype["Description"]:
                finalresult=drizzle_df["Rating"][index]
    elif currentmain=="Atmosphere" :
        #read atmospheredf to find which
        for index, desctype in atmosphere_df.iterrows():
            if currentdesc== desctype["Description"]:
                finalresult=atmosphere_df["Rating"][index]
    else:
        print(f'Staysafe during {currentmain}')
        finalresult="indoor"
                
#if outdoor check temperature and humidity
if finalresult=="outdoor":
    if (temp<7) or (temp>35) or (humidity>60):
        print(f'Staysafe, it is extreme temperature or humidity')
        finalresult="indoor"

print (finalresult)


Enjoy during Clouds
Staysafe, it is extreme temperature or humidity
indoor


In [10]:
#Find the names and addresses of nearbyplaces in given city,state entered by the user.
#Hint: See https://developers.google.com/places/web-service/supported_types
Acty = finalresult

if (Acty == "outdoor" or Acty == "Outdoor" or Acty == "OUTDOOR"):
    Acty = Acty_Outlist
elif (Acty == "indoor"or Acty == "Indoor" or Acty == "INDOOR"):
    Acty = Acty_Inlist

In [11]:
#Google maps nearby places search API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

places_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

In [12]:
type_of_place = []
place_names = []
addresses = []
rating = []
openorclose = []
#Noinfo = []
for i in range(len(Acty)):
    
    target_type = Acty[i]
    #radius = 40000

# Build URL using the Google Maps API and other fields
    #target_url = base_url+"key="+gkey+"&location="+lat+","+lng+"&radius="+str(radius)+"&type="+target_type#+"&opennow"
    target_url = base_url+"key="+gkey+"&location="+lat+","+lng+"&rankby=distance"+"&type="+target_type#+"&opennow"
    #print(target_url)
# Run request
    response = requests.get(target_url)
    city_placestogo = response.json()
    #print(city_placestogo)
#     place_names.append(target_type)
#     addresses.append(target_type)
#     rating.append(target_type)
#     openorclose.append(target_type)
    if city_placestogo["status"] == "ZERO_RESULTS":
        print(f'No {Acty[i]} near your place of choice')
        type_of_place.append(target_type)
        place_names.append("Not Available ")
        addresses.append("Not Available ")
        rating.append("Not Available ")
        openorclose.append("Not Available ")
        print("\n")
    else:
        print(f'Bingo!!! We have found good {Acty[i]}s near your place of choice!!!')
        
        #print("\n")
        #count = 1
        for item in city_placestogo["results"]:
            myplace = item["name"]
            myplace = myplace.replace(" ", "%20")
            target_url = places_url+"key="+gkey+"&input="+myplace+"&inputtype=textquery"+"&fields=formatted_address,name,rating,opening_hours"
            #print(target_url)
            response = requests.get(target_url)
            data = response.json()
            #print(data)
            type_of_place.append(target_type)
            try:
                place_names.append(data["candidates"][0]["name"])
            except:
                place_names.append(item["name"])
            try:    
                addresses.append(data["candidates"][0]["formatted_address"])
            except:
                addresses.append(item["vicinity"])
            try:    
                rating.append(data["candidates"][0]["rating"])
            except:   
                rating.append("NaN")
            try:    
                openorclose.append(data["candidates"][0]["opening_hours"]["open_now"])
            except:
                openorclose.append("Check the opening hours on the website")
                #print("No or incomplete information found")
            #count = count + 1
        print("\n")    

Bingo!!! We have found good aquariums near your place of choice!!!


Bingo!!! We have found good art_gallerys near your place of choice!!!


Bingo!!! We have found good bakerys near your place of choice!!!


Bingo!!! We have found good bars near your place of choice!!!


Bingo!!! We have found good book_stores near your place of choice!!!


Bingo!!! We have found good bowling_alleys near your place of choice!!!


Bingo!!! We have found good cafes near your place of choice!!!


No casino near your place of choice


Bingo!!! We have found good florists near your place of choice!!!


Bingo!!! We have found good gyms near your place of choice!!!


Bingo!!! We have found good librarys near your place of choice!!!


Bingo!!! We have found good lodgings near your place of choice!!!


Bingo!!! We have found good movie_rentals near your place of choice!!!


Bingo!!! We have found good movie_theaters near your place of choice!!!


Bingo!!! We have found good museums near your place of choice!!!


In [25]:
Df = pd.DataFrame({"Type":type_of_place,"Place":place_names,"Address":addresses, "Rating":rating,
                  "OpenorClose":openorclose})

In [26]:
Df[["Type","Place","Address","Rating","OpenorClose"]]

Type                                              Place  \
0         aquarium                               Water World Aquarium   
1         aquarium                Aquamarine The Fish & Aquarium Shop   
2         aquarium                                     The Fish Point   
3         aquarium                                      Fish Aquarium   
4         aquarium                              Unique Fish Aquariums   
5         aquarium                                   Pet Lovers Point   
6         aquarium              Fancy Fish Aqurium & Dog Kennal Store   
7         aquarium                                Sagar Fish Aquarium   
8         aquarium           Ocean World Adventure Park, Puerto Plata   
9         aquarium          Shree Ganesh Gift Toys & Stationery Point   
10        aquarium                             Aqua Palace Spa & Pool   
11        aquarium                         Shri Krishna Fish Aquarium   
12        aquarium                                   Swastik Aquarium   
13        aquarium                       Aquaking Service & Marketing   
14        aquarium                          Golden Fish Aquarium Shop   
15        aquarium                              Vikas Telecom Limited   
16        aquarium                              Friends Fish Aquarium   
17        aquarium                         Deva Glass & Fish Aquarium   
18        aquarium                          Baba Hardev Fish Aquarium   
19        aquarium                       Anand Fish And Aquarium Home   
20     art_gallery                              Ravi Varma Oleographs   
21     art_gallery                                        Art Gallery   
22     art_gallery                                Jain Hardware Store   
23     art_gallery                        ArtCenter College of Design   
24     art_gallery                                 Balaji Art Gallery   
25     art_gallery                                          Riya Arts   
26     art_gallery                             Majestic Art By Rajesh   
27     art_gallery                                 Ganesh Art Gallery   
28     art_gallery                                     Paintinghunter   
29     art_gallery                                Art & Craft Gallery   
..             ...                                                ...   
328  shopping_mall                                          New Delhi   
329  shopping_mall                                          Sector 15   
330  shopping_mall                                Komal Medical Store   
331  shopping_mall                            Riwaz collection Rohini   
332  shopping_mall                           Krishna Juice And Shakes   
333  shopping_mall                                          sumitkart   
334  shopping_mall  F22 DDA Market, Sector 3, Rohini New Delhi, Delhi   
335  shopping_mall                                 Aggarwal Mega Mart   
336  shopping_mall                                      K.K Garmernts   
337  shopping_mall                                  The Clothes Villa   
338            spa                         Laos female massage centre   
339            spa                                         Amrita Spa   
340            spa  Skin N Care Makeover Studio | Best Makeup Stud...   
341            spa                       Manju's the world of glamour   
342            spa                                      Womenstee spa   
343            spa                            Chacha Chunni Lal Hotel   
344            spa                                          Elide Spa   
345            spa                                    Sparsh Body Spa   
346            spa             Galaxy Wellbeing Spa - Massage Spa NYC   
347            spa                         Rose Beauty SPA And Saloon   
348            spa                                Pearl Spa and Sauna   
349            spa      Diamond Body Massage Parlour in Rohini, Delhi   
350            spa                                         Oleeva Spa   

In [51]:
Df.fillna(0, inplace=True)

dict={}

for item in Acty:
    item_Df = Df[Df["Type"]==item].reset_index()
    item_Df = item_Df.drop("index",1)       
    dict[item] = item_Df
    print(type(dict[item]))
    print(dict[item][["Place","Rating"]])
    print(dict[item]["Place"]) 
